In [1]:
import torch
from model import create_model
from dataloader import get_dataset, get_transform
import utils
import os
import lovasz_losses as L 
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dataset_dir = './dataset'
model_dir = './output'
num_epochs = 20

dataset_test = get_dataset("val", get_transform(train=False), dataset_dir)
data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=4, collate_fn=utils.collate_fn)

def criterion(inputs, target):
    sigmoid = torch.sigmoid(inputs['out'])
    sigmoid_aux = torch.sigmoid(inputs['aux'])
    preds = (inputs['out'].data > 0).long()

    loss = L.lovasz_softmax(sigmoid, target, classes=[1], ignore=128)
    loss_aux = L.lovasz_softmax(sigmoid_aux, target, classes=[1], ignore=128)

    iou = L.iou_binary(preds, target, ignore=128, per_image=True)

    return loss + 0.5 * loss_aux, iou

def evaluate(model, data_loader, device, epoch=None, writer=None, print_freq=1):
    model.eval()

    metric_logger = utils.MetricLogger(delimiter="  ")

    header = 'Test:'

    iou_list = []

    with torch.no_grad():
        for image, target in metric_logger.log_every(data_loader, print_freq, header):
            image, target = image.to(device), target.to(device)

            output = model(image)

            loss, iou = criterion(output, target)

            iou_list.append(iou)
            metric_logger.update(
                loss=loss.item(), iou=iou, mIOU=np.mean(iou_list))

            if writer is not None:
                writer.add_scalar('loss/test', loss.item(), epoch)
                writer.add_scalar('iou/test', iou, epoch)

    if writer is not None:
        writer.add_scalar('miou/test', np.mean(iou_list), epoch)

    print(f'{header} mIOU: {np.mean(iou_list)}')
    return np.mean(iou_list)

def evaluate_model(model_path):
    model = create_model(aux_loss=True, freeze_backbone=False)
    checkpoint = torch.load(model_path)
    model.load_state_dict(checkpoint['model'])
    model.to(device)
    model.eval()

    miou = evaluate(model, data_loader_test, device)
    return miou

best_miou = 0.0
best_model_path = None

for epoch in range(num_epochs):
    model_path = os.path.join(model_dir, f'model_{epoch}.pth')
    miou = evaluate_model(model_path)
    print(f'Epoch {epoch} - mIOU: {miou}')

    if miou > best_miou:
        best_miou = miou
        best_model_path = model_path

print(f'Best model path: {best_model_path} with mIOU: {best_miou}')


C:\Users\ozhyb\anaconda3\envs\lpsegmentation\lib\site-packages\torch\nn\modules\conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


Test:  [  0/201]  eta: 0:09:08  loss: 0.7024 (0.7024)  iou: 72.6315 (72.6315)  mIOU: 72.6315 (72.6315)  time: 2.7304  data: 2.2401  max mem: 964
Test:  [  1/201]  eta: 0:04:49  loss: 0.7024 (0.7053)  iou: 65.0957 (68.8636)  mIOU: 68.8636 (70.7476)  time: 1.4500  data: 1.1201  max mem: 964
Test:  [  2/201]  eta: 0:03:22  loss: 0.7083 (0.7450)  iou: 65.0957 (60.2985)  mIOU: 68.8636 (67.2645)  time: 1.0200  data: 0.7467  max mem: 969
Test:  [  3/201]  eta: 0:02:38  loss: 0.7083 (0.7713)  iou: 43.1681 (45.9985)  mIOU: 60.2985 (61.9480)  time: 0.8000  data: 0.5600  max mem: 969
Test:  [  4/201]  eta: 0:02:10  loss: 0.7613 (0.7693)  iou: 43.1681 (42.4071)  mIOU: 60.2985 (58.0398)  time: 0.6640  data: 0.4480  max mem: 969
Test:  [  5/201]  eta: 0:01:52  loss: 0.7083 (0.7585)  iou: 43.1681 (45.4330)  mIOU: 45.9985 (55.9387)  time: 0.5717  data: 0.3734  max mem: 969
Test:  [  6/201]  eta: 0:01:40  loss: 0.7613 (0.7721)  iou: 43.1681 (39.1656)  mIOU: 45.9985 (53.5425)  time: 0.5172  data: 0.3200